In [3]:
import numpy as np
from numpy.linalg import solve
import math
from sympy import Plane, Point3D, Line3D

In [4]:
def read_input(fname):
    data = []
    with open(fname, 'r') as inf:
        for line in inf.readlines():
            if line.strip() == '':
                continue
            p, s = line.strip().split(' @ ')
            data.append((tuple(int(v) for v in p.split(',')), tuple(int(v) for v in s.split(','))))

    return data

In [24]:
def n_crossing(data, minv, maxv):
    n = 0
    for i, (p, r) in enumerate(data):
        for j in range(i+1, len(data)):
            p = np.asarray(p[0:2])
            r = np.asarray(r[0:2])
        
            q = np.asarray(data[j][0][0:2])
            s = np.asarray(data[j][1][0:2])

            c1 = np.cross(q-p, s)
            c2 = np.cross(r, s)

            if c2 == 0:
                continue

            t = c1/c2

            res = p +t*r

            dist1 = res-p
            dist2 = res-q
            if minv<=res[0]<=maxv and minv<=res[1]<=maxv:
                if np.all(np.sign(dist1) == np.sign(r)) and np.all(np.sign(dist2) == np.sign(s)):
                    n += 1

    return n

In [28]:
def hail_hitter(data, first, second, third):

    A = np.array((6, 6))
    B = np.array(6)

    (p1x, p1y, p1z), (d1x, d1y, d1z) = data[first]
    (p2x, p2y, p2z), (d2x, d2y, d2z) = data[second]
    (p3x, p3y, p3z), (d3x, d3y, d3z) = data[third]
    A = np.array([[d1y-d2y, d2x-d1x, 0, p2y-p1y, p1x-p2x, 0],
                  [d1y-d3y, d3x-d1x, 0, p3y-p1y, p1x-p3x, 0],
                  [d1z-d2z, 0, d2x-d1x, p2z-p1z, 0, p1x-p2x],
                  [d1z-d3z, 0, d3x-d1x, p3z-p1z, 0, p1x-p3x],
                  [0, d1z-d2z, d2y-d1y, 0, p2z-p1z, p1y-p2y],
                  [0, d1z-d3z, d3y-d1y, 0, p3z-p1z, p1y-p3y]])
    B = np.array([p2y*d2x - p2x*d2y - p1y*d1x + p1x*d1y,
                  p3y*d3x - p3x*d3y - p1y*d1x + p1x*d1y,
                  p2z*d2x - p2x*d2z - p1z*d1x + p1x*d1z,
                  p3z*d3x - p3x*d3z - p1z*d1x + p1x*d1z,
                  p2z*d2y - p2y*d2z - p1z*d1y + p1y*d1z,
                  p3z*d3y - p3y*d3z - p1z*d1y + p1y*d1z])

    X = solve(A, B)

    # print(X)

    return np.round(np.sum(X[0:3]))

In [29]:
print('*****\nPuzzle1\n*****\n')

print('Test case\n')

nc = n_crossing(read_input('input24a.txt'), 7, 27)

print(f'N intersections is {nc}')

assert nc == 2

print('\nPuzzle case\n')

nc = n_crossing(read_input('input24.txt'), 200000000000000, 400000000000000)

print(f'N intersections is {nc}')

assert nc == 16018

print('\n*****\nPuzzle2\n*****\n')

print('Test case\n')

start = hail_hitter(read_input('input24a.txt'), 0, 1, 2)

print(f'start pos is {start}')

assert start == 47

print('\nPuzzle case\n')

start = hail_hitter(read_input('input24.txt'), 0, 1, 2)

print(f'start pos is {start}')

assert start == 1004774995964534

*****
Puzzle1
*****

Test case

N intersections is 2

Puzzle case

N intersections is 16018

*****
Puzzle2
*****

Test case

start pos is 47.0

Puzzle case

start pos is 1004774995964534.0
